In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2

import matplotlib.pyplot as plt
%matplotlib inline
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
import random
import gc

train_bbw_dir = '../input/training/training/bbw'
train_bbs_dir = '../input/training/training/bbs'
train_healthy_dir = '../input/training/training/healthy'

test_dir = '../input/test/test'

train_bbw = ['../input/training/training/bbw/{}'.format(i) for i in os.listdir(train_bbw_dir)]
train_healthy = ['../input/training/training/healthy/{}'.format(i) for i in os.listdir(train_healthy_dir)]
train_bbs = ['../input/training/training/bbs/{}'.format(i) for i in os.listdir(train_bbs_dir)]

test_imgs = ['../input/test/test/{}'.format(i) for i in os.listdir(test_dir)]

train_imgs = train_bbw + train_bbs + train_healthy
random.shuffle(train_imgs)

# Any results you write to the current directory are saved as output.

In [ ]:
import matplotlib.image as mpimg
for ima in train_imgs[0:3]:
    img=mpimg.imread(ima)
    imgplot = plt.imshow(img)
    plt.show()

In [ ]:
nrows = 227
ncolumns = 227
channels = 3
X = []
y = []

def read_and_process_image(list_of_images, label):
    
    for image in list_of_images:
        X.append(cv2.resize(cv2.imread(image,cv2.IMREAD_COLOR), (nrows,ncolumns), interpolation=cv2.INTER_CUBIC))
        y.append(label)
        
    return X, y

In [ ]:
X, y = read_and_process_image(train_bbw,0)
X, y = read_and_process_image(train_bbs,1)
X, y = read_and_process_image(train_healthy,2)

In [ ]:
plt.figure(figsize=(20,10))
columns = 5
for i in range(columns):
    plt.subplot(5 / columns + 1, columns, i+1)
    plt.imshow(X[i])

In [ ]:
import seaborn as sns
#Convert List to numpy array
X = np.array(X)
y = np.array(y)

# plotting the class labels
sns.countplot(y)
plt.title('Defined class labels')


In [ ]:
print("Shape of the training images is:", X.shape)
print("Shape of the labels is:", y.shape)

In [ ]:
#Splitting the training set into 80 : 20 ratio

from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.20, random_state = 2)
print("Shape of the train images is:", X_train.shape)
print("Shape of the validation images is:", X_val.shape)
print("Shape of the train labels is:", y_train.shape)
print("Shape of the validation labels is:", y_val.shape)


In [ ]:
# del X
# del y

# gc.collect()

#getting the validation and training data
ntrain = len(X_train)
nval = len(X_val)

# we will use a batch size of 32, Note : batch size should be in order of 2^n
batch_size = 32

In [ ]:
from keras import layers
from keras import models
from keras import optimizers

from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import img_to_array, load_img

In [ ]:
#Using Keras VGG-16 Model network architecture
model = models.Sequential()
model.add(layers.Conv2D(32,(3,3), activation = 'relu', input_shape= (227,227,3)))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64, (3,3), activation = 'relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(128, (3,3), activation = 'relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(128, (3,3), activation = 'relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Flatten())
model.add(layers.Dropout(0,5)) #Dropout for regularization
model.add(layers.Dense(512, activation = 'relu'))
model.add(layers.Dense(3, activation='softmax'))

model.summary()



In [ ]:

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range = 40, width_shift_range = 0.2, height_shift_range = 0.2, shear_range = 0.2, zoom_range =0.2, horizontal_flip = True)
val_datagen = ImageDataGenerator(rescale=1./255)


In [ ]:
from keras.utils import to_categorical
y_binary_train = to_categorical(y_train)
train_generator = train_datagen.flow(X_train, y_binary_train, batch_size = batch_size)
y_binary_val = to_categorical(y_val)
val_generator = val_datagen.flow(X_val, y_binary_val, batch_size = batch_size)


In [ ]:
history = model.fit_generator(train_generator, steps_per_epoch =ntrain//batch_size,epochs=128, validation_data=val_generator, validation_steps=nval//batch_size)

In [ ]:
model.save_weights('model_weights_new_128e.h5')
model.save('model_keras_new_128e.h5')


In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

#Train and validation accuracy
plt.plot(epochs, acc, 'b', label='Training accuracy')
plt.plot(epochs, val_acc, 'r', label='Validation Accuracy')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

#Train and validation loss
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and Validation loss')
plt.legend()

plt.show()


In [ ]:
X_test = test_imgs[0:20]
X = []
for image in X_test:
        X.append(cv2.resize(cv2.imread(image,cv2.IMREAD_COLOR), (nrows,ncolumns), interpolation=cv2.INTER_CUBIC))

x = np.array(X)

test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
i = 0;
text_labels = []
plt.figure(figsize=(30,20))

for batch in test_datagen.flow(x,batch_size=1):
    pred = model.predict(batch)
    if pred[0][0] > 0.5:
        text_labels.append('bbs')
    elif pred[0][1] > 0.5:
        text_labels.append('bbw')
    elif pred[0][2] > 0.5:
        text_labels.append('healthy')
    print(i)
    plt.subplot(5/columns +1 , columns, i+1)
    plt.title('This is a '+ text_labels[i])
    imgplot = plt.imshow(batch[0])
    i +=1
    if i/10 == 1:
        break
plt.show()
    
        